In [1]:
import os
from os.path import basename
from pathlib import Path

import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.plugins import CheckpointIO
from pytorch_lightning.utilities import rank_zero_only
from sconf import Config
import PIL.Image

from donut import DonutModel, DonutConfig
# import donut.model_custom
# from donut.model_custom import DonutModel, DonutConfig, BARTCustomTokenizer
from donut import DonutDataset
from lightning_module import DonutDataPLModule, DonutModelPLModule

import cv2
import numpy as np

In [2]:
config_path='/data/murayama/k8s/ocr_dxs1/donut/config/test_invoice.yaml'
config=Config(config_path)
model = DonutModel.from_pretrained(
    config.pretrained_model_name_or_path,
    input_size=config.input_size,
    max_length=config.max_length,
    align_long_axis=config.align_long_axis,
    enable_char_map=config.char_map,
    box_pred=config.get('box_pred',False)
)

if 'classes' in config:
    stokens = []
    for cn in config.classes:
        stokens += [fr"<s_{cn}>", fr"</s_{cn}>"]
    model.decoder.add_special_tokens(stokens)

model = model.cuda().half().eval()


/usr/local/lib/python3.8/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
image_path='/data/murayama/k8s/ocr_dxs1/donut/dataset/kie/invoice_nttd/test_imgs/202307_nttd_001.pdf-1.png'
image=PIL.Image.open(image_path)
# outs = model.inference(image, prompt='<s_health>', return_attentions=True, return_confs=True, return_tokens=True)
try:
    outs = model.inference_custom(image, prompt='<s_invoice_nttd>')
except Exception as ex:
    import traceback
    traceback.print_exc()

In [6]:
outs["predictions"]

{'invoice': {'content': [{'Account_Name': {'content': 'フリコミサキ ダロウ',
     'box': [0.0, 0.0, 0.0, 0.0, 0.0],
     'score': 1.0}},
   {'Account_Number': {'content': '1234123',
     'box': [0.0, 0.0, 0.0, 0.0, 0.0],
     'score': 1.0}},
   {'Account_Type': {'content': '普通',
     'box': [0.0, 0.0, 0.0, 0.0, 0.0],
     'score': 1.0}},
   {'Bank_Name': {'content': '三井住友銀行',
     'box': [0.0, 0.0, 0.0, 0.0, 0.0],
     'score': 1.0}},
   {'Bankbranch_Name': {'content': 'テスト支店',
     'box': [0.0, 0.0, 0.0, 0.0, 0.0],
     'score': 1.0}},
   {'Biller_Address': {'content': '東京都港区 1-1',
     'box': [0.0, 0.0, 0.0, 0.0, 0.0],
     'score': 1.0}},
   {'Biller_Company': {'content': '株式会社請求元',
     'box': [0.0, 0.0, 0.0, 0.0, 0.0],
     'score': 1.0}},
   {'Billing_Company': {'content': '株式会社請求先',
     'box': [0.0, 0.0, 0.0, 0.0, 0.0],
     'score': 1.0}},
   {'Due_Date': {'content': '2023/6/23',
     'box': [0.0, 0.0, 0.0, 0.0, 0.0],
     'score': 1.0}},
   {'Invoice_Number': {'content': '123122',
   

In [10]:
print(model.decoder.model.config.vocab_size)
print(model.decoder.tokenizer.vocab_size)

57583
57522
